In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import time
import datetime
import dateutil.relativedelta

import gc
import warnings

warnings.simplefilter(action='ignore')

pd.set_option('display.width',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_info_columns', 200)

In [38]:
def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    return df

def sub_month_lag(x):
    return x['purchase_date_first'] - dateutil.relativedelta.relativedelta(months=x['month_lag_first'])

def get_r(history):
    agg_func = {
        'month_lag': ['first', ],
        'purchase_date': ['first', ],
        }
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    agg_history['reference_date'] = agg_history.apply(sub_month_lag, axis=1)
    agg_history['reference_date'] = agg_history['reference_date'].apply(lambda x: x + dateutil.relativedelta.relativedelta(day=1, months=+1, days=-1))

    agg_history.drop(columns=['month_lag_first', 'purchase_date_first'], inplace=True)

    return agg_history

def binarize(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

def date2date(start, end):
    return (start.dt.date - end.dt.date).dt.days

def summary(df):
    stats = []
    for col in df.columns:
        stats.append((col, df[col].nunique(), 
                      df[col].isnull().sum() * 100 / df.shape[0], 
                      df[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                      df[col].dtype))
    
    return pd.DataFrame(stats, columns=['feature', 'unique', 'missing', 'mode', 'type'])

In [6]:
historical_transactions = pd.read_csv('./data/historical_transactions.csv', parse_dates=['purchase_date'])
new_transactions = pd.read_csv('./data/new_merchant_transactions.csv', parse_dates=['purchase_date'])

In [7]:
historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [8]:
np.percentile(historical_transactions['purchase_amount'], [1, 5, 50, 95, 99])

array([-0.74324133, -0.73819242, -0.68834948, -0.16861249,  1.22084097])

In [9]:
np.percentile(new_transactions['purchase_amount'], [1, 5, 50, 95, 99])

array([-0.74239984, -0.73638924, -0.67484064, -0.08574128,  1.4628174 ])

In [10]:
historical_transactions['purchase_amount'] = historical_transactions['purchase_amount'].apply(lambda x: min(x, 1.5))

In [11]:
new_transactions['purchase_amount'] = new_transactions['purchase_amount'].apply(lambda x: min(x, 1.5))

In [12]:
historical_transactions['category_2'] = historical_transactions['category_2'].fillna(-1, )
new_transactions['category_2'] = new_transactions['category_2'].fillna(-1, )

historical_transactions['category_3'] = historical_transactions['category_3'].map({'A':0, 'B':1, 'C':2})
new_transactions['category_3'] = new_transactions['category_3'].map({'A':0, 'B':1, 'C':2})

historical_transactions['category_3'] = historical_transactions['category_3'].fillna(-1, )
new_transactions['category_3'] = new_transactions['category_3'].fillna(-1, )

historical_transactions['merchant_id'] = historical_transactions['merchant_id'].fillna('-1', )
new_transactions['merchant_id'] = new_transactions['merchant_id'].fillna('-1', )

In [13]:
train = read_data('./data/train.csv')
test = read_data('./data/test.csv')
target = train['target']

In [14]:
test[test['card_id']=='C_ID_c27b4f80f7']

,first_active_month,card_id,feature_1,feature_2,feature_3
11578,NaT,C_ID_c27b4f80f7,5,2,1


In [15]:
test['first_active_month'][test['card_id']=='C_ID_c27b4f80f7'] = pd.to_datetime(datetime.date(2017, 3, 1))

In [16]:
train_a = train[['card_id', 'first_active_month']]
test_a = test[['card_id', 'first_active_month']]

In [17]:
tmp_a = pd.concat([train_a, test_a]).drop_duplicates().reset_index(drop=True)

In [18]:
historical_transactions = pd.merge(historical_transactions, tmp_a, on='card_id', how='left')
new_transactions = pd.merge(new_transactions, tmp_a, on='card_id', how='left')

In [19]:
historical_transactions['purchase_date'] = pd.to_datetime(historical_transactions['purchase_date'])
new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])

In [20]:
hist_r = get_r(historical_transactions)

In [21]:
historical_transactions = pd.merge(historical_transactions, hist_r, on='card_id', how='left')
new_transactions = pd.merge(new_transactions, hist_r, on='card_id', how='left')

In [22]:
binary_func = ['mean', ]

category_func = ['nunique', ]

contiguous_func = ['mean', 'median', 'max', 'min', 'std', ]

In [23]:
agg_fun = {
    'authorized_flag': binary_func, 
}
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

train = pd.merge(train, auth_mean, on='card_id', how='left')
test = pd.merge(test, auth_mean, on='card_id', how='left')

In [24]:
for df in [historical_transactions, new_transactions]:
    df['a2p'] = date2date(df['first_active_month'], df['purchase_date'])
    df['p2r'] = date2date(df['purchase_date'], df['reference_date'])
    
    df['p_vs_m'] = df['purchase_amount'] / (df['month_lag'].abs()+1)
    
    df["installments"].replace(-1, np.NaN, inplace=True)
    df["installments"].replace(999, np.NaN, inplace=True)
    
    df['p_vs_i'] = df['purchase_amount'] / (df['installments'].abs()+1)

    df['quarter'] = df['purchase_date'].dt.quarter

    df['month'] = df['purchase_date'].dt.month
    
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour

    df['is_month_start'] = (df.purchase_date.dt.is_month_start).astype(int)

    df['weekend'] = (df.purchase_date.dt.weekday>=5).astype(int)  

In [25]:
def aggregate_transactions_hist(history):
        
    agg_func = {
        'is_month_start': binary_func, 
        'weekend': binary_func,
        'category_1': binary_func,
        #
        'category_2': category_func, #
        'category_3': category_func, #
        
        'state_id': ['nunique', ],
        'city_id': ['nunique', ],
        'subsector_id': ['nunique', ],
        'merchant_category_id': ['nunique', ],
        'merchant_id': ['nunique', ],
        
        'quarter': category_func,
        'month': category_func, 
        'weekofyear': category_func,
        'dayofweek': category_func,
        'day': category_func,
        'hour': category_func,
        #
        'a2p': contiguous_func, 
        'p2r': contiguous_func, 
        
        'month_lag': contiguous_func,
        'purchase_amount': ['sum', 'mean', 'median', 'max', 'min', 'std', ], 
        'installments': contiguous_func, 
        'p_vs_m': contiguous_func,
        'p_vs_i': contiguous_func,
        'purchase_date': ['max', 'min'],
        'first_active_month': ['first'],
        'reference_date': ['first'],
        }
            
    for col in ['category_2', 'category_3', 
                'state_id', 'city_id', 'subsector_id', 'merchant_category_id', 'merchant_id', 
                'quarter', 
                'month', 'weekofyear', 
                'dayofweek',
                'day',
                'hour'
               ]:
        
        history[col+'_p_mean'] = history.groupby([col])['purchase_amount'].transform('mean')  # mean encode
        agg_func[col+'_p_mean'] = contiguous_func

    agg_history = history.groupby(['card_id']).agg(agg_func)
    
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    agg_history['first_year'] = agg_history['first_active_month_first'].dt.year
    agg_history['first_quarter'] = agg_history['first_active_month_first'].dt.quarter
    agg_history['first_month'] = agg_history['first_active_month_first'].dt.month
    
    agg_history['re_year'] = agg_history['reference_date_first'].dt.year
    agg_history['re_quarter'] = agg_history['reference_date_first'].dt.quarter
    agg_history['re_month'] = agg_history['reference_date_first'].dt.month
        
    agg_history['a2r'] = date2date(agg_history['first_active_month_first'], agg_history['reference_date_first'])

    agg_history.drop(columns=['first_active_month_first', 'reference_date_first'], inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

83

In [26]:
%%time
history = aggregate_transactions_hist(historical_transactions)
    
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]

history['hist_p2p'] = (history['hist_purchase_date_max'] - history['hist_purchase_date_min']).dt.days
history['hist_sleep'] = history['hist_p2p'] - history['hist_day_nunique']

###
history['hist_p2p_vs_count'] = history['hist_p2p']/history['hist_transactions_count']
history['hist_sleep_vs_count'] = history['hist_sleep']/history['hist_transactions_count']

history['hist_p2r_vs_count'] = history['hist_p2r_min'].abs()/history['hist_transactions_count']

###
history['hist_count_vs_p2p'] = history['hist_transactions_count']/(history['hist_p2p'].abs()+1)
history['hist_sleep_vs_p2p'] = history['hist_sleep']/(history['hist_p2p'].abs()+1)
history['hist_p_vs_p2p'] = history['hist_purchase_amount_sum']/(history['hist_p2p'].abs()+1)
###
history['hist_count_vs_p2r'] = history['hist_transactions_count']/(history['hist_p2r_min'].abs()+1)
history['hist_sleep_vs_p2r'] = history['hist_sleep']/(history['hist_p2r_min'].abs()+1)
history['hist_p_vs_p2r'] = history['hist_purchase_amount_sum']/(history['hist_p2r_min'].abs()+1)
###

CPU times: user 9min 21s, sys: 43.7 s, total: 10min 5s
Wall time: 3min 16s


In [27]:
history.head()

,card_id,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_first_year,hist_first_quarter,hist_first_month,hist_re_year,hist_re_quarter,hist_re_month,hist_a2r,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_p2r_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,hist_count_vs_p2r,hist_sleep_vs_p2r,hist_p_vs_p2r
0,C_ID_00007093c1,149,0.067114,0.167785,0.187919,3,2,3,4,13,18,29,4,12,39,7,28,18,-197.838926,-185.0,-13,-391,104.216256,-194.161074,-207.0,-1,-379,104.216256,-5.852349,-6.0,0,-12,3.453114,-76.852110,-0.515786,-0.587627,1.500000,-0.728876,0.297818,1.288591,1.0,6.0,1.0,0.764900,-0.128251,-0.085235,0.214286,-0.721363,0.149473,-0.251977,-0.290808,0.375000,-0.364438,0.128706,2018-02-27 05:14:57,2017-02-14 14:00:43,-0.572437,-0.582086,-0.530200,-0.597130,0.020423,-0.538263,-0.617830,-0.123849,-0.617830,0.182199,-0.567946,-0.576530,-0.530200,-0.594657,0.018279,-0.568970,-0.577454,-0.525830,-0.599463,0.021061,-0.587132,-0.619453,-0.107234,-0.683504,0.116425,-0.599078,-0.620200,-0.344769,-0.698723,0.095077,-0.586346,-0.574567,-0.311478,-0.699339,0.095052,-0.596635,-0.598183,-0.592935,-0.598229,0.002129,-0.597112,-0.597791,-0.590924,-0.602054,0.002974,-0.596773,-0.5

In [28]:
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

del history; gc.collect()

14

In [29]:
def aggregate_transactions_new(new):
        
    agg_func = {
        'is_month_start': binary_func, 
        'weekend': binary_func,
        'category_1': binary_func,
        #
        
        'category_2': category_func, #
        'category_3': category_func, #
        
        'state_id': ['nunique', ],
        'city_id': ['nunique', ],
        'subsector_id': ['nunique', ],
        'merchant_category_id': ['nunique', ],
        'merchant_id': ['nunique', ],
        
        'quarter': category_func,
        'month': category_func, 
        'weekofyear': category_func,
        'dayofweek': category_func,
        'day': category_func,
        'hour': category_func,
        
        #
        'a2p': contiguous_func, 
        'p2r': contiguous_func, 
        
        'month_lag': contiguous_func,
        'purchase_amount': ['sum', 'mean', 'median', 'max', 'min', 'std', ], 
        'installments': contiguous_func, 
        'p_vs_m': contiguous_func,
        'p_vs_i': contiguous_func,
        'purchase_date': ['max', 'min'],
        }
            
    for col in ['category_2', 'category_3', 
                'state_id', 'city_id', 'subsector_id', 'merchant_category_id', 'merchant_id', 
                'quarter', 
                'month', 'weekofyear', 
                'dayofweek',
                'day',
                'hour'
               ]:
        
        new[col+'_p_mean'] = new.groupby([col])['purchase_amount'].transform('mean')  # mean encode

        agg_func[col+'_p_mean'] = contiguous_func

    agg_new = new.groupby(['card_id']).agg(agg_func)
    
    agg_new.columns = ['_'.join(col).strip() for col in agg_new.columns.values]
    agg_new.reset_index(inplace=True)
        
    df = (new.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_new = pd.merge(df, agg_new, on='card_id', how='left')
        
    return agg_new

0

In [30]:
%%time
new = aggregate_transactions_new(new_transactions)
    
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]
    
new['new_p2p'] = (new['new_purchase_date_max'] - new['new_purchase_date_min']).dt.days
new['new_sleep'] = new['new_p2p'] - new['new_day_nunique']
###
new['new_p2p_vs_count'] = new['new_p2p']/new['new_transactions_count']
new['new_sleep_vs_count'] = new['new_sleep']/new['new_transactions_count']

new['new_p2r_vs_count'] = new['new_p2r_max'].abs()/new['new_transactions_count']

###
new['new_count_vs_p2p'] = new['new_transactions_count']/(new['new_p2p'].abs()+1)
new['new_sleep_vs_p2p'] = new['new_sleep']/(new['new_p2p'].abs()+1)
new['new_p_vs_p2p'] = new['new_purchase_amount_sum']/(new['new_p2p'].abs()+1)
###
new['new_count_vs_p2r'] = new['new_transactions_count']/(new['new_p2r_max'].abs()+1)
new['new_sleep_vs_p2r'] = new['new_sleep']/(new['new_p2r_max'].abs()+1)
new['new_p_vs_p2r'] = new['new_purchase_amount_sum']/(new['new_p2r_max'].abs()+1)
###

CPU times: user 3min 3s, sys: 10.2 s, total: 3min 13s
Wall time: 21.2 s


In [31]:
new.head()

,card_id,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_mean,new_installments_median,new_installments_max,new_installments_min,new_installments_std,new_p_vs_m_mean,new_p_vs_m_median,new_p_vs_m_max,new_p_vs_m_min,new_p_vs_m_std,new_p_vs_i_mean,new_p_vs_i_median,new_p_vs_i_max,new_p_vs_i_min,new_p_vs_i_std,new_purchase_date_max,new_purchase_date_min,new_category_2_p_mean_mean,new_category_2_p_mean_median,new_category_2_p_mean_max,new_category_2_p_mean_min,new_category_2_p_mean_std,new_category_3_p_mean_mean,new_category_3_p_mean_median,new_category_3_p_mean_max,new_category_3_p_mean_min,new_category_3_p_mean_std,new_state_id_p_mean_mean,new_state_id_p_mean_median,new_state_id_p_mean_max,new_state_id_p_mean_min,new_state_id_p_mean_std,new_city_id_p_mean_mean,new_city_id_p_mean_median,new_city_id_p_mean_max,new_city_id_p_mean_min,new_city_id_p_mean_std,new_subsector_id_p_mean_mean,new_subsector_id_p_mean_median,new_subsector_id_p_mean_max,new_subsector_id_p_mean_min,new_subsector_id_p_mean_std,new_merchant_category_id_p_mean_mean,new_merchant_category_id_p_mean_median,new_merchant_category_id_p_mean_max,new_merchant_category_id_p_mean_min,new_merchant_category_id_p_mean_std,new_merchant_id_p_mean_mean,new_merchant_id_p_mean_median,new_merchant_id_p_mean_max,new_merchant_id_p_mean_min,new_merchant_id_p_mean_std,new_quarter_p_mean_mean,new_quarter_p_mean_median,new_quarter_p_mean_max,new_quarter_p_mean_min,new_quarter_p_mean_std,new_month_p_mean_mean,new_month_p_mean_median,new_month_p_mean_max,new_month_p_mean_min,new_month_p_mean_std,new_weekofyear_p_mean_mean,new_weekofyear_p_mean_median,new_weekofyear_p_mean_max,new_weekofyear_p_mean_min,new_weekofyear_p_mean_std,new_dayofweek_p_mean_mean,new_dayofweek_p_mean_median,new_dayofweek_p_mean_max,new_dayofweek_p_mean_min,new_dayofweek_p_mean_std,new_day_p_mean_mean,new_day_p_mean_median,new_day_p_mean_max,new_day_p_mean_min,new_day_p_mean_std,new_hour_p_mean_mean,new_hour_p_mean_median,new_hour_p_mean_max,new_hour_p_mean_min,new_hour_p_mean_std,new_p2p,new_sleep,new_p2p_vs_count,new_sleep_vs_count,new_p2r_vs_count,new_count_vs_p2p,new_sleep_vs_p2p,new_p_vs_p2p,new_count_vs_p2r,new_sleep_vs_p2r,new_p_vs_p2r
0,C_ID_00007093c1,2,0.000000,0.000000,0.000000,2,1,2,2,2,2,2,1,1,2,2,2,2,-429.000000,-429.0,-426,-432,4.242641,37.000000,37.0,40,34,4.242641,2.000000,2.0,2,2,0.000000,-1.328524,-0.664262,-0.664262,-0.656749,-0.671775,0.010625,1.00,1.0,1.0,1.0,0.000000,-0.221421,-0.221421,-0.218916,-0.223925,0.003542,-0.332131,-0.332131,-0.328374,-0.335888,0.005313,2018-04-09 16:23:59,2018-04-03 11:13:35,-0.579960,-0.579960,-0.571085,-0.588835,0.012551,-0.615461,-0.615461,-0.615461,-0.615461,0.000000,-0.579101,-0.579101,-0.571887,-0.586315,0.010202,-0.576606,-0.576606,-0.575068,-0.578145,0.002176,-0.508819,-0.508819,-0.445344,-0.572294,0.089767,-0.500311,-0.500311,-0.459415,-0.541207,0.057836,-0.620281,-0.620281,-0.568788,-0.671775,0.072823,-0.572793,-0.572793,-0.572793,-0.572793,0.000000,-0.571095,-0.571095,-0.571095,-0.571095,0.000000,-0.571103,-0.571103,-0.569963,-0.572242,0.001612,-0.562547,-0.562547,-0.561972,-0.563122,0.000813,-0.572118,-0.572118,-0.570274,-0.573962,0.002608,-0.558911,-0.558911,-0.555792,-0.562029,0.004411,6,4,3.000000,2.000000,20.000000,0.285714,0.571429,-0.189789,0.048780,0.097561

In [32]:
train = pd.merge(train, new, on='card_id', how='left')
test = pd.merge(test, new, on='card_id', how='left')

del new; gc.collect()

28

In [33]:
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1
train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [34]:
for df in [train, test]:
    ###
    df['c_p2p_diff'] = df['hist_p2p_vs_count'] - df['new_p2p_vs_count']
    df['c_sleep_diff'] = df['hist_sleep_vs_count'] - df['new_sleep_vs_count']
    df['c_p_diff'] = df['hist_purchase_amount_mean'] - df['new_purchase_amount_mean']
    ###
    df['p2p_count_diff'] = df['hist_count_vs_p2p'] - df['new_count_vs_p2p']
    df['p2p_sleep_diff'] = df['hist_sleep_vs_p2p'] - df['new_sleep_vs_p2p']
    df['p2p_p_diff'] = df['hist_p_vs_p2p'] - df['new_p_vs_p2p']
    ###
    df['p2r_count_diff'] = df['hist_count_vs_p2r'] - df['new_count_vs_p2r']
    df['p2r_sleep_diff'] = df['hist_sleep_vs_p2r'] - df['new_sleep_vs_p2r']
    df['p2r_p_diff'] = df['hist_p_vs_p2r'] - df['new_p_vs_p2r']
    ###
    df['c_p2p_diff_vs'] = df['c_p2p_diff'] / df['hist_p2p_vs_count']
    df['c_sleep_diff_vs'] = df['c_sleep_diff'] / df['hist_sleep_vs_count']
    df['c_p_diff_vs']  = df['c_p_diff'] / df['hist_purchase_amount_mean']
    ###
    df['p2p_count_diff_vs'] = df['p2p_count_diff'] / df['hist_count_vs_p2p']
    df['p2p_sleep_diff_vs'] = df['p2p_sleep_diff'] / df['hist_sleep_vs_p2p']
    df['p2p_p_diff_vs']  = df['p2p_p_diff'] / df['hist_p_vs_p2p']
    ###
    df['p2r_count_diff_vs'] = df['p2r_count_diff'] / df['hist_count_vs_p2r']
    df['p2r_sleep_diff_vs'] = df['p2r_sleep_diff'] / df['hist_sleep_vs_p2r']
    df['p2r_p_diff_vs']  = df['p2r_p_diff'] / df['hist_p_vs_p2r']
    ###  
    ###
    for f in ['hist_purchase_date_max','hist_purchase_date_min', 
              'new_purchase_date_max', 'new_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9

In [35]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,authorized_flag_mean,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_first_year,hist_first_quarter,hist_first_month,hist_re_year,hist_re_quarter,hist_re_month,hist_a2r,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_p2r_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,hist_count_vs_p2r,hist_sleep_vs_p2r,hist_p_vs_p2r,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_mean,new_installments_median,new_ins

In [36]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,authorized_flag_mean,hist_transactions_count,hist_is_month_start_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_nunique,hist_category_3_nunique,hist_state_id_nunique,hist_city_id_nunique,hist_subsector_id_nunique,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_quarter_nunique,hist_month_nunique,hist_weekofyear_nunique,hist_dayofweek_nunique,hist_day_nunique,hist_hour_nunique,hist_a2p_mean,hist_a2p_median,hist_a2p_max,hist_a2p_min,hist_a2p_std,hist_p2r_mean,hist_p2r_median,hist_p2r_max,hist_p2r_min,hist_p2r_std,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_max,hist_month_lag_min,hist_month_lag_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_mean,hist_installments_median,hist_installments_max,hist_installments_min,hist_installments_std,hist_p_vs_m_mean,hist_p_vs_m_median,hist_p_vs_m_max,hist_p_vs_m_min,hist_p_vs_m_std,hist_p_vs_i_mean,hist_p_vs_i_median,hist_p_vs_i_max,hist_p_vs_i_min,hist_p_vs_i_std,hist_purchase_date_max,hist_purchase_date_min,hist_category_2_p_mean_mean,hist_category_2_p_mean_median,hist_category_2_p_mean_max,hist_category_2_p_mean_min,hist_category_2_p_mean_std,hist_category_3_p_mean_mean,hist_category_3_p_mean_median,hist_category_3_p_mean_max,hist_category_3_p_mean_min,hist_category_3_p_mean_std,hist_state_id_p_mean_mean,hist_state_id_p_mean_median,hist_state_id_p_mean_max,hist_state_id_p_mean_min,hist_state_id_p_mean_std,hist_city_id_p_mean_mean,hist_city_id_p_mean_median,hist_city_id_p_mean_max,hist_city_id_p_mean_min,hist_city_id_p_mean_std,hist_subsector_id_p_mean_mean,hist_subsector_id_p_mean_median,hist_subsector_id_p_mean_max,hist_subsector_id_p_mean_min,hist_subsector_id_p_mean_std,hist_merchant_category_id_p_mean_mean,hist_merchant_category_id_p_mean_median,hist_merchant_category_id_p_mean_max,hist_merchant_category_id_p_mean_min,hist_merchant_category_id_p_mean_std,hist_merchant_id_p_mean_mean,hist_merchant_id_p_mean_median,hist_merchant_id_p_mean_max,hist_merchant_id_p_mean_min,hist_merchant_id_p_mean_std,hist_quarter_p_mean_mean,hist_quarter_p_mean_median,hist_quarter_p_mean_max,hist_quarter_p_mean_min,hist_quarter_p_mean_std,hist_month_p_mean_mean,hist_month_p_mean_median,hist_month_p_mean_max,hist_month_p_mean_min,hist_month_p_mean_std,hist_weekofyear_p_mean_mean,hist_weekofyear_p_mean_median,hist_weekofyear_p_mean_max,hist_weekofyear_p_mean_min,hist_weekofyear_p_mean_std,hist_dayofweek_p_mean_mean,hist_dayofweek_p_mean_median,hist_dayofweek_p_mean_max,hist_dayofweek_p_mean_min,hist_dayofweek_p_mean_std,hist_day_p_mean_mean,hist_day_p_mean_median,hist_day_p_mean_max,hist_day_p_mean_min,hist_day_p_mean_std,hist_hour_p_mean_mean,hist_hour_p_mean_median,hist_hour_p_mean_max,hist_hour_p_mean_min,hist_hour_p_mean_std,hist_first_year,hist_first_quarter,hist_first_month,hist_re_year,hist_re_quarter,hist_re_month,hist_a2r,hist_p2p,hist_sleep,hist_p2p_vs_count,hist_sleep_vs_count,hist_p2r_vs_count,hist_count_vs_p2p,hist_sleep_vs_p2p,hist_p_vs_p2p,hist_count_vs_p2r,hist_sleep_vs_p2r,hist_p_vs_p2r,new_transactions_count,new_is_month_start_mean,new_weekend_mean,new_category_1_mean,new_category_2_nunique,new_category_3_nunique,new_state_id_nunique,new_city_id_nunique,new_subsector_id_nunique,new_merchant_category_id_nunique,new_merchant_id_nunique,new_quarter_nunique,new_month_nunique,new_weekofyear_nunique,new_dayofweek_nunique,new_day_nunique,new_hour_nunique,new_a2p_mean,new_a2p_median,new_a2p_max,new_a2p_min,new_a2p_std,new_p2r_mean,new_p2r_median,new_p2r_max,new_p2r_min,new_p2r_std,new_month_lag_mean,new_month_lag_median,new_month_lag_max,new_month_lag_min,new_month_lag_std,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_median,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_mean,new_installments_median,new_installmen

In [39]:
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

Train Shape: (201917, 295)
Test Shape: (123623, 293)


In [40]:
train_summary = summary(train)
test_summary = summary(test)

In [41]:
train_summary.sort_values('unique', ascending=False)

,feature,unique,missing,mode,type
1,card_id,201917,0.000000,0.000495,object
144,hist_p_vs_p2r,201897,0.000000,0.000991,float64
111,hist_weekofyear_p_mean_std,201890,0.000000,0.000991,float64
126,hist_hour_p_mean_std,201866,0.000000,0.007429,float64
141,hist_p_vs_p2p,201832,0.000000,0.003962,float64
54,hist_p_vs_m_std,201822,0.000000,0.005448,float64
50,hist_p_vs_m_mean,201821,0.000000,0.003962,float64
122,hist_hour_p_mean_mean,201778,0.000000,0.001486,float64
107,hist_weekofyear_p_mean_mean,201736,0.000000,0.002972,float64
55,hist_p_vs_i_mean,201668,0.000000,0.011886,float64


In [42]:
test_summary.sort_values('unique', ascending=False)

,feature,unique,missing,mode,type
1,card_id,123623,0.000000,0.000809,object
110,hist_weekofyear_p_mean_std,123614,0.000000,0.002427,float64
143,hist_p_vs_p2r,123613,0.000000,0.001618,float64
125,hist_hour_p_mean_std,123605,0.000000,0.004853,float64
140,hist_p_vs_p2p,123587,0.000000,0.004045,float64
49,hist_p_vs_m_mean,123585,0.000000,0.004045,float64
53,hist_p_vs_m_std,123583,0.000000,0.003236,float64
121,hist_hour_p_mean_mean,123557,0.000000,0.002427,float64
106,hist_weekofyear_p_mean_mean,123554,0.000000,0.003236,float64
54,hist_p_vs_i_mean,123497,0.000000,0.016178,float64


In [65]:
train.to_csv("./data/pre_train_clip.csv")

In [66]:
test.to_csv("./data/pre_test_clip.csv")